install library if not

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.5 MB/s eta 0:00:00


In [2]:
!pip install fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 78.7 MB/s eta 0:00:00


In [4]:
!pip install mapclassify matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.4 MB/s eta 0:00:00


Import necessary library

In [5]:
import streamlit as st
import numpy as np
import pandas as pd
import geopandas as gpd
import mapclassify
import matplotlib.pyplot as plt
import folium
import fiona
import pyproj
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.geometry import box
from collections import OrderedDict
from google.colab import files

Hardcoded constant value.

In [6]:
# Create the data dictionary
data = {
    'SN': range(1, 26),
    'scientific_name': ['Abies spp', 'Acacia catechu', 'Adina cardifolia', 'Albizia spp', 'Alnus nepalensis',
                       'Anogeissus latifolia', 'Bombax ceiba', 'Cedrela toona', 'Dalbergia sissoo',
                       'Eugenia Jambolana', 'Hymenodictyon excelsum', 'Lagerstroemia parviflora',
                       'Michelia champaca', 'Pinus roxburghii', 'Pinus wallichiana', 'Quercus spp',
                       'Schima wallichii', 'Shorea robusta', 'Terminalia alata', 'Trewia nudiflora',
                       'Tsuga spp', 'Terai spp', 'Hill spp', 'Coniferious', 'Broadleaved'],
    'a': [-2.4453, -2.3256, -2.5626, -2.4284, -2.7761, -2.272, -2.3856, -2.1832, -2.1959, -2.5693,
          -2.585, -2.3411, -2.0152, -2.977, -2.8195, -2.36, -2.7385, -2.4554, -2.4616, -2.4585,
          -2.5293, -2.3993, -2.3204, np.nan, np.nan],
    'b': [1.722, 1.6476, 1.8598, 1.7609, 1.9006, 1.7499, 1.7414, 1.8679, 1.6567, 1.8816,
          1.9437, 1.7246, 1.8555, 1.9235, 1.725, 1.968, 1.8155, 1.9026, 1.8497, 1.8043,
          1.7815, 1.7836, 1.8507, np.nan, np.nan],
    'c': [1.0757, 1.0552, 0.8783, 0.9662, 0.9428, 0.9174, 1.0063, 0.7569, 0.9899, 0.8498,
          0.7902, 0.9702, 0.763, 1.0019, 1.1623, 0.7496, 1.0072, 0.8352, 0.88, 0.922,
          1.0369, 0.9546, 0.8223, np.nan, np.nan],
    'a1': [5.4433, 5.4401, 5.4681, 4.4031, 6.019, 4.9502, 4.5554, 4.9705, 4.358, 5.1749,
           5.5572, 5.3349, 3.3499, 6.2696, 5.7216, 4.8511, 7.4617, 5.2026, 4.5968, 5.3475,
           5.2774, 4.8991, 5.5323, np.nan, np.nan],
    'b1': [-2.6902, -2.491, -2.491, -2.2094, -2.7271, -2.3353, -2.3009, -2.3436, -2.1559, -2.3636,
           -2.496, -2.4428, -2.0161, -2.8252, -2.6788, -2.4494, -3.0676, -2.4788, -2.2305, -2.4774,
           -2.6483, -2.3406, -2.4815, np.nan, np.nan],
    's': [0.436, 0.443, 0.443, 0.443, 0.803, 0.443, 0.443, 0.443, 0.684, 0.443,
          0.443, 0.443, 0.443, 0.189, 0.683, 0.747, 0.52, 0.055, 0.443, 0.443,
          0.443, 0.443, 0.443, 0.436, 0.443],
    'm': [0.372, 0.511, 0.511, 0.511, 1.226, 0.511, 0.511, 0.511, 0.684, 0.511,
          0.511, 0.511, 0.511, 0.256, 0.488, 0.96, 0.186, 0.341, 0.511, 0.511,
          0.511, 0.511, 0.511, 0.372, 0.511],
    'bg': [0.355, 0.71, 0.71, 0.71, 1.51, 0.71, 0.71, 0.71, 0.684, 0.71,
           0.71, 0.71, 0.71, 0.3, 0.41, 1.06, 0.168, 0.357, 0.71, 0.71,
           0.71, 0.71, 0.71, 0.355, 0.71],
    'Local_Name': ['Thingre Salla', 'Khayar', 'Karma', 'Siris', 'Uttis', 'Banjhi', 'Simal', 'Tooni',
                   'Sissoo', 'Jamun', 'Bhudkul', 'Botdhayero', 'Chanp', 'Khote Salla', 'Gobre Salla',
                   'Kharsu', 'Chilaune', 'Sal', 'Saj', 'Gamhari', 'Dhupi Salla', 'Terai Spp',
                   'Hill spp', '', '']
}

wrap constant data in a variable

In [7]:
sppVal = pd.DataFrame(data)

#Upload Stem mapping csv file using streamlit library

In [8]:
'''# Replace the file upload code
uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    # ... (rest of your code that uses df)
else:
    st.write("Please upload tree location or stem mapping CSV file.")'''

2024-11-08 12:04:52.007 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-08 12:04:52.009 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-08 12:04:52.011 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-08 12:04:52.022 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-08 12:04:52.238 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-08 12:04:52.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-08 12:04:52.244 Thread 'MainThread'

For testing purpose to upload file I am using geopandas. when using streamlit comment this code.

In [9]:
stemmapping = files.upload()
df = pd.read_csv('TreeLoc.csv')

Saving TreeLoc.csv to TreeLoc.csv


Read sppVal.csv table inorder to join dataframe with its species value

# Join the GeoDataFrames based on the 'species' column
merged_gdf = df_with_remarks.merge(sppVal, left_on='species', right_on='Scientific Name')

In [10]:
joined_df = df.merge(sppVal, left_on='species', right_on='scientific_name')

copy the joined_df as 'result_df'.

In [11]:
result_df = joined_df.copy()

## Function to perform calculations and add new columns

In [ ]:
def add_calculated_columns(df):
    df['stem_volume'] = np.exp(df['a'] + df['b'] * np.log(df['dia_cm']) + df['c'] * np.log(df['height_m'])) / 1000
    df['branch_ratio'] = df['dia_cm'].apply(lambda x: 0.1 if x < 10 else 0.2)
    df['branch_volume'] = df['stem_volume'] * df['branch_ratio']
    df['tree_volume'] = df['stem_volume'] + df['branch_volume']
    df['cm10diaratio'] = np.exp(df['a1'] + df['b1'] * np.log(df['dia_cm']))
    df['cm10topvolume'] = df['stem_volume'] * df['cm10diaratio']
    df['gross_volume'] = df['stem_volume'] - df['cm10topvolume']
    df['net_volume'] = df.apply(lambda row: row['gross_volume'] * 0.9 if row['class'] == 'A' else row['gross_volume'] * 0.8, axis=1)
    df['net_volum_cft'] = df['net_volume'] * 35.3147
    df['firewood_m3'] = df['tree_volume'] - df['net_volume']
    df['firewood_chatta'] = df['firewood_m3'] * 0.105944
    return df

Apply the function to the dataframe

In [ ]:
result_df = add_calculated_columns(df=result_df)

In [ ]:
columns_to_drop = ['SN', 'scientific_name', 'a', 'b', 'c', 'a1', 'b1', 's', 'm', 'bg']
result_df = result_df.drop(columns=columns_to_drop)

Download the result using streamlit instead of other library.

In [ ]:
result_df.to_csv('result_df.csv', index=False)
files.download('result_df.csv')

Download result using streamlit library.

In [ ]:
# ... (your existing code) ...

# Download button
@st.cache_data
def convert_df(df):
    # IMPORTANT: Cache the conversion to prevent computation on every rerun
    return df.to_csv(index=False).encode('utf-8')

csv = convert_df(result_df)

st.download_button(
    label="Download data as CSV",
    data=csv,
    file_name='result_df.csv',
    mime='text/csv',
)

#convert pandas dataframe to geopandas dataframe

In [ ]:
result_df['geometry'] = result_df.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)

In [ ]:
result_gdf = gpd.GeoDataFrame(result_df, geometry='geometry', crs='epsg:4326')

In [ ]:
result_gdf.explore()

In [ ]:
'''# @title Spatial Distribution of Tree Species
# Assuming your data is in a pandas DataFrame called 'df'
plt.figure(figsize=(10, 8))
for species in df['species'].unique():
    subset = df[df['species'] == species]
    plt.scatter(subset['LONGITUDE'], subset['LATITUDE'], label=species, alpha=0.6, s=2)  # Adjust 's' for marker size

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Distribution of Tree Species')
_ = plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Species')  # Place legend outside the plot'''

# Create Bounding box withing tree map extent and will be used to generate grid of twenty meter

In [ ]:
# Get the total bounds (xmin, ymin, xmax, ymax)
xmin, ymin, xmax, ymax = result_gdf.total_bounds
# Create a Polygon using the bounds
bounding_polygon = box(xmin, ymin, xmax, ymax)
# Optionally, create a GeoDataFrame with the bounding polygon
bounding_gdf = gpd.GeoDataFrame(geometry=[bounding_polygon], crs=result_gdf.crs)

#in this code user need to enter tree spacinge float value using streamlit library. for convinience purpose here geopandas library is used.

In [ ]:
# Assuming 'bounding_gdf' is already defined and is a GeoDataFrame.
# Also assuming you have a way to get user input from a textbox (e.g., using a GUI library).

# Replace this with your actual method of getting user input.
user_input_spacing = input("Enter grid spacing in meters (default is 20m): ")

try:
    grid_spacing = float(user_input_spacing)  # Try to convert input to float
except ValueError:
    grid_spacing = 20  # Use default if input is invalid

# Now use grid_spacing to create your grid using bounding_gdf
# Example (replace with actual code):

# ... your code to create the grid using grid_spacing and bounding_gdf ...

Create grid

In [ ]:
# Assuming 'bounding_gdf' is your GeoDataFrame in 'EPSG:4326'

# Define the grid spacing in meters (20 meters in this case)
spacing_meters = 100

# Get the bounds of the bounding geometry
xmin, ymin, xmax, ymax = bounding_gdf.total_bounds

# Create a pyproj transformer to convert meters to degrees
transformer = pyproj.Transformer.from_crs("EPSG:4326", bounding_gdf.crs, always_xy=True)

# Calculate spacing in degrees based on the center of the bounding box
center_lat = (ymin + ymax) / 2
spacing_degrees = spacing_meters / (111320 * np.cos(np.deg2rad(center_lat)))  # Approximate conversion

# Create a list of x and y coordinates for the grid points
x_coords = np.arange(xmin, xmax, spacing_degrees)
y_coords = np.arange(ymin, ymax, spacing_degrees)

# Create a list of polygons representing the grid cells
polygons = []
for x in x_coords:
    for y in y_coords:
        polygons.append(Polygon([(x, y), (x + spacing_degrees, y),
                                 (x + spacing_degrees, y + spacing_degrees), (x, y + spacing_degrees)]))

# Create a GeoDataFrame from the polygons
grid_gdf = gpd.GeoDataFrame({'geometry': polygons}, crs='EPSG:4326')

# (Optional) Clip the grid to the bounding geometry
grid_gdf = gpd.clip(grid_gdf, bounding_gdf)

In [ ]:
grid_gdf.explore()

In [ ]:
# ... (your existing code to create grid_gdf and result_gdf) ...

# Perform a spatial join to find intersecting polygons, but keep only the grid cell index
intersected_grid_indices = gpd.sjoin(grid_gdf, result_gdf, how='inner', predicate='intersects').index.unique()

# Select the unique grid cells from grid_gdf based on the indices
selected_polygons_gdf = grid_gdf[grid_gdf.index.isin(intersected_grid_indices)]

# (Optional) Reset index
selected_polygons_gdf = selected_polygons_gdf.reset_index(drop=True)

# ... (further processing) ...

In [ ]:
selected_polygons_gdf.explore()

#create Bounding box

#Creating grid and centroid of the grid

In [ ]:
# Get the bounds of your bounding GeoDataFrame
xmin, ymin, xmax, ymax = bounding_gdf.total_bounds

# Define the grid cell size in degrees (approximate for 20 meters)
# Note: This is a rough approximation and might not be perfectly accurate for all latitudes
cell_size_degrees = 20 / 111320  # Roughly 1 degree = 111.32 km, so 20 meters ≈ 20/111320 degrees

# Create the grid points
xcoords = np.arange(xmin, xmax, cell_size_degrees)
ycoords = np.arange(ymin, ymax, cell_size_degrees)

# Create a list of grid polygons
grid_cells = []
for x in xcoords:
    for y in ycoords:
        polygon = Polygon([(x, y), (x + cell_size_degrees, y),
                           (x + cell_size_degrees, y + cell_size_degrees),
                           (x, y + cell_size_degrees)])
        grid_cells.append(polygon)

# Create a GeoDataFrame for the grid
grid_gdf = gpd.GeoDataFrame(geometry=grid_cells, crs=bounding_gdf.crs)

# Clip the grid to the extent of your bounding GeoDataFrame
grid_gdf = gpd.clip(grid_gdf, bounding_gdf.geometry.iloc[0])

#creating centroid gdf

In [ ]:
# Create a GeoDataFrame with the centroids of the grid cells
centroid_gdf = selected_polygons_gdf.copy()  # Create a copy to avoid modifying grid_gdf
centroid_gdf['geometry'] = centroid_gdf['geometry'].centroid

# (Optional) Reset index if needed
centroid_gdf = centroid_gdf.reset_index(drop=False) #keep the original index

In [ ]:
centroid_gdf.explore()

In [ ]:
# ... (your existing code to create centroid_gdf and result_gdf) ...

# Spatial join to find nearest tree for each centroid
# Using 'nearest' predicate to find the closest tree
joined_gdf = gpd.sjoin_nearest(centroid_gdf, result_gdf, how='left', distance_col='distance')

# Group by centroid index and get the index of the nearest tree
nearest_tree_indices = joined_gdf.groupby(joined_gdf.index)['distance'].idxmin()

# Create 'remark' column in result_gdf and initialize with 'non mother tree'
result_gdf['remark'] = 'non mother tree'

# Set 'remark' to 'mother Tree' for the nearest trees
result_gdf.loc[nearest_tree_indices, 'remark'] = 'mother Tree'

# ... (further processing) ...

The following code needs user input like ('EPSG:32644' or 'EPSG:32645' and so on) This input should use streamlit library. for testing purpose it is being used geopandas library.

In [ ]:
# Assuming 'centroid_gdf' and 'result_gdf' are your GeoDataFrames in 'EPSG:4326'

# Get user input for projected CRS
projected_crs = input("Enter the desired projected CRS (e.g., EPSG:32633): ")

try:
    # Reproject the GeoDataFrames
    centroid_gdf_proj = centroid_gdf.to_crs(projected_crs)
    result_gdf_proj = result_gdf.to_crs(projected_crs)

    # Perform the spatial join in the projected CRS
    joined_gdf = gpd.sjoin_nearest(centroid_gdf_proj, result_gdf_proj, how='left', distance_col='distance')

    # ... (rest of your code to update 'remark' column) ...

except pyproj.exceptions.CRSError as e:
    print(f"Error: Invalid CRS provided. Please enter a valid EPSG code. \nDetails: {e}")

In [ ]:
# Create the interactive map
m = result_gdf.explore(column='remark', legend=False)  # Remove default legend

# Define a style function to set colors and size based on 'remark'
def custom_style_function(feature):
    if feature['properties']['remark'] == 'mother Tree':
        return {'fillColor': 'red', 'color': 'red', 'radius': 5}  # Red, smaller size
    else:
        return {'fillColor': 'green', 'color': 'green', 'radius': 1}  # Green, smaller size

# Apply the custom style function
for _, layer in m._children.items():
    if isinstance(layer, folium.features.GeoJson):
        layer.style_function = custom_style_function

# Calculate the counts of each tree type
mother_tree_count = result_gdf[result_gdf['remark'] == 'mother Tree'].shape[0]
other_tree_count = result_gdf[result_gdf['remark'] != 'mother Tree'].shape[0]

# Create a simple HTML legend using CSS for colored circles and counts
legend_html = f"""
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 150px; height: 120px;
            border:1px solid grey; z-index:9999; font-size:15px;
            background-color:white; opacity: 0.90; padding: 10px;">
<b>Legend</b><br>
<div style="display: flex; align-items: center;">
    <div style="width: 10px; height: 10px; background-color: red; border-radius: 50%; margin-right: 5px;"></div> Mother Tree ({mother_tree_count})
</div>
<div style="display: flex; align-items: center;">
    <div style="width: 10px; height: 10px; background-color: green; border-radius: 50%; margin-right: 5px;"></div> Other Trees ({other_tree_count})
</div>
</div>
"""
# Add the legend to the map
m.get_root().html.add_child(folium.Element(legend_html))
# Display the map
m

#the following code uses geopandas library but it needs to be downloaded using streamlit library. correct the code accordingly.

In [ ]:
result_gdf.to_csv('result_gdf.csv', index=False)
files.download('result_gdf.csv')

In [ ]:
# Assuming `result_gdf` is your GeoDataFrame
result_gdf.to_file('result_gdf.kml', driver='KML')

# Download the file
files.download('result_gdf.kml')

In [ ]:
!pip install fiona

In [ ]:
schema = {
    'geometry': 'Point',
    'properties': OrderedDict([
        # ... your desired columns and data types ...
        ('TID', 'int'),
        ('species', 'str'),
        ('LONGITUDE', 'float'),
        ('LATITUDE', 'float'),
        ('dia_cm', 'float'),
        ('height_m', 'float'),
        ('class', 'float'),
        ('Local_Name', 'str'),
        ('stem_volume', 'float'),
        ('branch_ratio', 'float'),
        ('branch_volume', 'float'),
        ('tree_volume', 'float'),
        ('cm10diaratio', 'float'),
        ('cm10topvolume', 'float'),
        ('gross_volume', 'float'),
        ('net_volume', 'float'),
        ('net_volum_cft', 'float'),
        ('firewood_m3', 'float'),
        ('firewood_chatta', 'float'),
        ('remark', 'str'),


        # ... add other columns and their data types ...
    ]),
}

with fiona.open('result_gdf.kml', 'w', driver='KML', crs=result_gdf.crs, schema=schema) as c:
    for index, row in result_gdf.iterrows():
        c.write({
            'geometry': row.geometry.__geo_interface__,
            'properties': {
                # ... your desired columns and values ...
                'TID': row['TID'],
                'species': row['species'],
                'LONGITUDE': row['LONGITUDE'],
                'LATITUDE': row['LATITUDE'],
                'dia_cm': row['dia_cm'],
                'height_m': row['height_m'],
                'class': row['class'],
                'Local_Name': row['Local_Name'],
                'stem_volume': row['stem_volume'],
                'branch_ratio': row['branch_ratio'],
                'branch_volume': row['branch_volume'],
                'tree_volume': row['tree_volume'],
                'cm10diaratio': row['cm10diaratio'],
                'cm10topvolume': row['cm10topvolume'],
                'gross_volume': row['gross_volume'],
                'net_volume': row['net_volume'],
                'net_volum_cft': row['net_volum_cft'],
                'firewood_m3': row['firewood_m3'],
                'firewood_chatta': row['firewood_chatta'],
                'remark': row['remark'],

                # ... add other columns and their corresponding values ...
            }
        })

files.download('result_gdf.kml')